# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
import sys
import torch
# Custom library paths
sys.path.extend(['../', './scr'])

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import requests
import collections
import time
# from scr.game import predict_next_character
from scr.model import RNN  # Assuming RNN is defined in this module
from scr.base_model import BaseModel  # Import the ModelCheckpointManager
from collections import Counter
from scr.game import guess_character

model_class = RNN

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def get_access_token_from_file(file_path):
    with open(file_path, 'r') as file:
        return file.readline().strip()

# Read the access token from file
access_token = get_access_token_from_file("hangman_api_creds.txt")

class HangmanAPI(object):
    def __init__(self, model_class=model_class, model_path='/models/trial_None/default_rnn.pth', \
        device=device, access_token=access_token, session=None, timeout=2000):

        self.access_token = access_token
        self.timeout = timeout

        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        # self.repetitive_pred_counter = Counter()
        self.device = device

        full_dictionary_location = "data/words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.char_frequency = collections.Counter("".join(self.full_dictionary))

        self.max_word_length = len(max(self.full_dictionary, key=len))
        self.loaded_model = self.load_model_once(model_class, model_path, device)

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']
        data = {link: 0 for link in links}
        for link in links:
            for _ in range(10):
                start_time = time.time()
                requests.get(link)
                data[link] += time.time() - start_time
        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        return f'{link}/trexsim/hangman'

    def build_dictionary(self, file_location):
        with open(file_location, 'r') as file:
            return [line.strip() for line in file]

    def load_model_once(self, model_class, model_path, device):
        return BaseModel.load_model(model_class, model_path, device)


    def guess(self, word):
        cleaned_word = "".join(char.lower() for char in word if \
            char.isalpha() or char == '_')

        # Predict the next character using the updated guess_character function
        guessed_char = guess_character(
            self.loaded_model, cleaned_word,
            self.char_frequency,
            self.max_word_length,
            self.device,
            self.guessed_letters  # Pass the list of guessed letters
        )

        # Add the new guess to the guessed letters list
        if guessed_char not in self.guessed_letters:
            self.guessed_letters.append(guessed_char)
        
        return guessed_char

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            # print(word)
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)

                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", \
                        "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                    
                status = res.get('status')
                tries_remains = res.get('tries_remains')

                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [7]:
# Create an instance of HangmanAPI
api = HangmanAPI(model_class, device, \
    access_token, timeout=2000)

AttributeError: 'torch.device' object has no attribute 'seek'. You can only torch.load from a file that is seekable. Please pre-load the data into a buffer like io.BytesIO and try to load from it instead.

In [ ]:
# api = HangmanAPI(access_token=access_token, timeout=2000)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [ ]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

Successfully start a new game! Game ID: 2db2070cba38. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
Guessing letter: z
Sever response: {'game_id': '2db2070cba38', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ '}
Guessing letter: b
Sever response: {'game_id': '2db2070cba38', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ _ _ _ _ '}
Guessing letter: u
Sever response: {'game_id': '2db2070cba38', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ u _ _ _ '}
Guessing letter: c
Sever response: {'game_id': '2db2070cba38', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ c u _ _ _ '}
Guessing letter: f
Sever response: {'game_id': '2db2070cba38', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ c u _ _ _ '}
Guessing letter: e
Sever response: {'game_id': '2db2070cba38', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ c u _ e _ '}
Guessing letter: i
Sever response: {'game_id': '2db2070cba38', 'status': 'ongoing', 'tries_

In [ ]:
# for i in range(1000):
#     print('Playing', i, 'th game')
#     success = api.start_game(practice=1, verbose=True)
#     if success:
#         print("Won the game!")
#     else:
#         print("Lost the game!")
#     time.sleep(0.5)  # DO NOT REMOVE

# [total_practice_runs, total_recorded_runs, total_recorded_successes, total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
# practice_success_rate = total_practice_successes / total_practice_runs
# print('Ran %d practice games out of an allotted 100,000. Practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

In [ ]:
STOP

NameError: name 'STOP' is not defined

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
# for i in range(1000):
#     print('Playing ', i, ' th game')
#     # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
#     #api.start_game(practice=0,verbose=False)
    
#     # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
#     time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
# [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
# success_rate = total_recorded_successes/total_recorded_runs
# print('overall success rate = %.3f' % success_rate)